In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.cloud import bigquery as bq
from google.oauth2 import service_account

import pandas as pd
import plotly.graph_objects as go
import json

import queries
import utils

# Config 

In [3]:
config = json.load(open("config.json", "r"))
credentials = config["credentials"]
project = config["project_id"]
dataset = config["dataset_id"]
credentials = service_account.Credentials.from_service_account_file(
            credentials,
            scopes=["https://www.googleapis.com/auth/cloud-platform"])


In [4]:
client = bq.Client(project=project,
                  credentials=credentials)

# Stat desc EPO full text data

[parseEPO]:(https://cverluise.github.io/parseEPO/)
This analysis is based on the EPO "full-text data for text analytics dataset". 

After a bit of processing (see [parseEPO][parseEPO]), we are able to load it in a BigQuery table (`<project>.<dataset>.full_text`). 

<details>
    
<b>BQ EPO full text table schema </b>:    
   
|Field name	|Type	|Mode	|Description|
|---|---|---|---|    
|publication_number	|STRING	|NULLABLE	|DOCDB publication number|
|publication_date	|DATE	|NULLABLE	|Publication date of the EP patent|
|url	|RECORD	|NULLABLE	|Url link to the pdf of the EP patent|
|url. language|	STRING	|NULLABLE	|Language of the pdf|
|url. text	|STRING	|NULLABLE|	Url|
|title	|RECORD	|NULLABLE	|Title of the patent|
|title. language|	STRING|	REPEATED|	Title language|
|title. text|	STRING|	REPEATED|	Localized title|
|abstract|	RECORD|	NULLABLE|	Abstract of the patent|
|abstract. language|	STRING|	NULLABLE|	Abstract language|
|abstract. text|	STRING|	NULLABLE|	Localized abstract|
|description|	RECORD|	NULLABLE|	Description of the patent|
|description. language|	STRING|	NULLABLE|	Language of the description|
|description. text|	STRING|	NULLABLE|	Localized description|
|claims|	RECORD|	NULLABLE|	Claims| of patent|
|claims. language|	STRING|	REPEATED|	Claims language|
|claims. text|	STRING|	REPEATED|	Localized claims|
|amendment|	RECORD|	NULLABLE|	Amendments|
|amendment. language|	STRING|	REPEATED|	Amendments language|
|amendment. text|	STRING|	REPEATED|	Localized amendments|
    
</details>    

The `full_text` table is relatively large due to full-text data. We create an intermediary table with high-level descriptive statistics which is much lighter (`epo_fulltext_desc`). Hence, we can investigate descriptive statistics relative to the dataset without any budget risk.

**`epo_fulltext_desc` preview**

|family_id|publication_number|publication_date|has_title|title_language|has_abstract|abstract_language|has_description|description_language|has_claims|claims_language|has_amendment|amendment_language|has_url|url_language|	
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|41351794|EP-2462853-A2|2012-06-13|true|de|true|it|true|it|true|en|false|true|it|

In [5]:
destination = client.dataset(dataset).table("epo_fulltext_desc")
job_config = bq.QueryJobConfig(destination=destination)
job = queries.safe_query(queries.query_epo_desc_table(f"{project}.{dataset}.full_text"), 
             client, 
             job_config=job_config)

# safe_query is asimple wrapper for BQ jobs. It returns an Error message when the destination table defined 
# in job_config already exists and does not execute the query.
# The goal is to avoid large queries generating tables which _already_ exist

⚠ JOB ERROR: duplicate. This kind of error is usually raised when the
destination table already exists. If you want to replace the table, execute the
same function with job_config.write_disposition='WRITE_TRUNCATE'. The query will
process 229Gb. Otherwise, just use the existing target table. table.


## EP data over time

In [6]:
df_famid = client.query(queries.query_nb_famid_epo(f"{project}.{dataset}.epo_fulltext_desc")).to_dataframe()
df_pubnum = client.query(queries.query_nb_pubnum_epo(f"{project}.{dataset}.epo_fulltext_desc")).to_dataframe()

In [7]:
fig = utils.plot_series(pd.merge(df_famid, df_pubnum, on="year").set_index("year"))
fig.write_image("plots/nb_famid_pubnum_epo.png")

![](plots/nb_famid_pubnum_epo.png)

In [37]:
print("Nb pubnum:", df_pubnum.sum()["nb_pubnum"])
print("Nb famid:", df_famid.sum()["nb_family"])

Nb pubnum: 5938057
Nb famid: 3285768


## EP full text data coverage

> We work at the family level
> - A family is assigned to the year of it first appearance in the EP full text-patent dataset. This date migt differ from the priority year
> - A family `has_*` **True** if at least one of the patents in the family has a `*` (e.g. `*`= `abstract`)

### Family level - non null data - all languages

> We consider all languages (en, fr, de, it)

In [8]:
# Number of families by year of 1st apperance in EP with text data 
query = queries.query_nb_full_text_epo(f"{project}.{dataset}.epo_fulltext_desc")
df_desc_epo = client.query(query).to_dataframe()

In [9]:
# Number of families by year of 1st apperance in EP
query = queries.query_nb_famid_epo(f"{project}.{dataset}.epo_fulltext_desc")
df_nb_famid_epo = client.query(query).to_dataframe()

In [10]:
# Share of family with text data, by year
df = pd.merge(df_desc_epo, df_nb_famid_epo, on="year").set_index("year")
for var in df.filter(regex="has").columns:
    df[var.replace("nb", "share")]=df[var]/df["nb_family"]

In [11]:
for v in ["share", "nb"]:
    fig = utils.plot_series(df.filter(regex=v))
    fig.write_image(f"plots/{v}_fulltext_epo.png")

Nb of families with text data | Share of families with text data 
:-------------------------:|:-------------------------:
![](./plots/nb_fulltext_epo.png)  |  ![](./plots/share_fulltext_epo.png)

In [12]:
# All time aggregate
tmp = df.sum()
out = {"1978-2019":{}}
for var in df.filter(regex="nb_has").columns:
    share = round(tmp[var]/tmp["nb_family"], 2)
    out["1978-2019"].update({var.replace("nb", "share"): share})
pd.DataFrame.from_dict(out, orient='index')

,share_has_title,share_has_abstract,share_has_description,share_has_claims,share_has_amendment,share_has_url
1978-2019,0.6,0.2,0.33,0.33,0.0,0.41


### Family level - non null data - english only

> We restrict to english only (en)

In [13]:
query = queries.query_nb_full_text_epo(f"{project}.{dataset}.epo_fulltext_desc",
                                      english_bool=True)
df_desc_epo_en = client.query(query).to_dataframe()

In [14]:
df_en = pd.merge(df_desc_epo_en, df_nb_famid_epo, on="year").set_index("year")
for var in df_en.filter(regex="has").columns:
    df_en[var.replace("nb", "share")]=df_en[var]/df_en["nb_family"]

In [15]:
for v in ["share", "nb"]:
    fig = utils.plot_series(df_en.filter(regex=v))
    fig.write_image(f"plots/{v}_fulltext_epo_en.png")

Nb of families with text data | Share of families with text data 
:-------------------------:|:-------------------------:
![](./plots/nb_fulltext_epo_en.png)  |  ![](./plots/share_fulltext_epo_en.png)

In [16]:
# All time aggregate - en only
tmp = df_en.sum()
out = {"1978-2019":{}}
for var in df_en.filter(regex="nb_has").columns:
    share = round(tmp[var]/tmp["nb_family"], 2)
    out["1978-2019"].update({var.replace("nb", "share"): share})
pd.DataFrame.from_dict(out, orient='index')    

,share_has_title,share_has_abstract,share_has_description,share_has_claims,share_has_amendment,share_has_url
1978-2019,0.22,0.02,0.22,0.22,0.0,0.22


**Baseline: 22% of patent families in the EPO full text database have an english description.**

# Wordlwide coverage

Patents which are in the same (simple) family have the same text components. This means that any patent text can be propagated to all patents in the same family. Thanks to that, EP (resp US) full text data can cover patents well beyond the EP (resp US) office. 

Next, we look at the implicit worldwide coverage of EP and US full-text patents.

> We focus on the description - all languages

## EP full-text data

In [17]:
query = queries.query_nb_fam_with_desc_from_epo(f"{project}.{dataset}.epo_fulltext_desc")
df_num = client.query(query).to_dataframe()

In [18]:
query = queries.query_nb_fam_cnt_yr
df_denom = client.query(query).to_dataframe()

In [19]:
df_coverage_epo = utils.compute_coverage(df_num, df_denom)
# Note: the mean_coverage is just a mean of mean - ie, all years are equally weighted

In [20]:
df_coverage_epo.tail(2)

year,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,mean_coverage
country_code,,,,,,,,,,,,,,,,,,,,,
PT,0.050317,0.182222,0.359126,0.399038,0.431576,0.471735,0.478189,0.490744,0.466362,0.513210,...,0.458034,0.503116,0.487617,0.499735,0.488795,0.476820,0.491869,0.499468,0.417541,0.438721
EP,0.794643,0.758818,0.705703,0.701681,0.707730,0.725521,0.719600,0.717313,0.708447,0.685287,...,0.417305,0.405435,0.393628,0.382566,0.359637,0.344194,0.309729,0.273310,0.169127,0.525174


In [21]:
fig = utils.plot_heatmap(df_coverage_epo.iloc[-15:, :-1]) # we focus on the top 15 and we skip the mean_coverage
fig.write_image("plots/coverage_epo.png")

![](plots/coverage_epo.png)

In [22]:
fig = utils.plot_bars(df_coverage_epo.reset_index().iloc[-30:,:])
fig.write_image("plots/mean_coverage_epo.png")
# Note: just a mean of mean.

## BigQuery patents-public-data full text data

In [23]:
destination = client.dataset(dataset).table("nonepo_fulltext_famid")
job_config = bq.QueryJobConfig(destination=destination)
job = queries.safe_query(queries.query_famid_nonepo, 
             client, 
             job_config=job_config)

⚠ JOB ERROR: duplicate. This kind of error is usually raised when the
destination table already exists. If you want to replace the table, execute the
same function with job_config.write_disposition='WRITE_TRUNCATE'. The query will
process 733Gb. Otherwise, just use the existing target table. table.


In [24]:
query = queries.query_nb_fam_with_desc_from_nonepo(f"{project}.{dataset}.nonepo_fulltext_famid")
df_num = client.query(query).to_dataframe()

In [25]:
df_coverage_nonepo = utils.compute_coverage(df_num, df_denom)

In [26]:
df_coverage_nonepo.tail(2)

year,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,mean_coverage
country_code,,,,,,,,,,,,,,,,,,,,,
CA,0.635405,0.648613,0.653665,0.655862,0.658823,0.654687,0.658533,0.668010,0.666255,0.678527,...,0.869699,0.871182,0.871803,0.872161,0.878359,0.880573,0.855718,0.75079,0.807615,0.756748
US,0.998488,0.998018,0.999131,0.999152,0.999054,0.998850,0.998920,0.999223,0.999496,0.999496,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,1.00000,1.000000,0.999718


In [27]:
fig = utils.plot_heatmap(df_coverage_nonepo.iloc[-15:, :-1]) # we focus on the top 15 and we skip the mean_coverage
fig.write_image("plots/coverage_nonepo.png")

![](plots/coverage_nonepo.png)

In [28]:
fig = utils.plot_bars(df_coverage_nonepo.reset_index().iloc[-30:,:])
fig.write_image("plots/mean_coverage_nonepo.png")
# Note: just a mean of mean.

## BigQuery patents-public-data augmented from EPO full text data

> Worldwide coverage from families with full-text data from either US or EP full-text dataset

In [29]:
destination = client.dataset(dataset).table("full_en_fulltext_famid")
job_config = bq.QueryJobConfig(destination=destination)
job = queries.safe_query(queries.query_famid_en_fulltext(f"{project}.{dataset}.epo_fulltext_desc",
                                                  f"{project}.{dataset}.nonepo_fulltext_famid"), 
             client, 
             job_config=job_config)

⚠ JOB ERROR: duplicate. This kind of error is usually raised when the
destination table already exists. If you want to replace the table, execute the
same function with job_config.write_disposition='WRITE_TRUNCATE'. The query will
process 0Gb. Otherwise, just use the existing target table. table.


In [30]:
query = queries.query_nb_fam_with_desc_from_nonepo(f"{project}.{dataset}.full_en_fulltext_famid")
df_num = client.query(query).to_dataframe()

In [31]:
df_coverage_augmented = utils.compute_coverage(df_num, df_denom)

In [32]:
df_coverage_augmented.tail(2)

year,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,mean_coverage
country_code,,,,,,,,,,,,,,,,,,,,,
CA,0.635405,0.648654,0.654337,0.663075,0.677895,0.683654,0.698053,0.713269,0.720587,0.729441,...,0.884016,0.886098,0.886467,0.883615,0.887574,0.890051,0.861394,0.755147,0.81328,0.779960
US,0.998488,0.998059,0.999131,0.999152,0.999071,0.998915,0.998920,0.999237,0.999496,0.999496,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,1.000000,1.00000,0.999723


In [33]:
fig = utils.plot_heatmap(df_coverage_augmented.iloc[-15:, :-1]) # we focus on the top 15 and we skip the mean_coverage
fig.write_image("plots/coverage_augmented.png")

![](plots/coverage_augmented.png)

In [34]:
fig = utils.plot_bars(df_coverage_augmented.reset_index().iloc[-30:,:])
fig.write_image("plots/mean_coverage_augmented.png")
# Note: just a mean of mean.

**Baseline, EP full-text data adds ~175 000 families to the set of patent families with english full-text description.**